In [6]:
import pandas as pd
import re
import numpy as np

def reg_func(s):
    if re.search(r'user_\d+', s):
        return re.search(r'user_\d+', s).group(0)
    else:
        return ''
    
def reg_time(s):
    if s != np.nan:
        return re.sub('\[', '', s)
    else:
        return s

In [7]:
log = pd.read_csv('log.csv', header=None)
log.columns = ['user_id', 'time', 'bet', 'win']

log['user_id'] = log['user_id'].apply(lambda x: reg_func(x))

In [9]:
users = pd.read_csv('users.csv', sep = '\t', encoding = 'koi8_r')
users.columns = ['user_id', 'email', 'geo']

In [14]:
log.time.isna().sum()

15

#### Удаление пропусков
Пропуски можно удалять автоматически. Во многих случаях это правильно, так как данные с большим количеством пропусков часто не имеют смысла и не приносят никакой пользы.  

Удалять данные с пропусками можно с помощью метода dropna().

Параметр axis в методе dropna() говорит методу, по какой оси удалять значения.  

1. Если нужно удалить строки, в которых встречается пропуск (NaN), следует указать axis=0.  Зачем это делать? Например, у нас из 1000 примеров данных про пользователей пропуски есть в пяти. Разумно их удалить, так как их количество пренебрежимо мало.
2. Если нужно удалить столбцы, в которых встречается пропуск (NaN), нужно указывать axis=1. Зачем? Иногда в одном конкретном столбце пропусков настолько много, что с ними просто не хочется возиться - смысла в них все равно почти нет.

Еще один интересный параметр - subset. Что он делает? Если передать в него список значений по одной оси (например, названия столбцов) и задать при этом в параметре axis другую ось (в нашем случае 0), то мы удалим те строки, для которых в данных столбцах находится пропуск. То же самое работает и наоборот: нужно поменять axis на 1 и вместо названий столбцов передавать индексы строк.

Перед удалением строк обязательно сделайте бэкап.

In [16]:
log_nonans = log.dropna(axis = 1)
log_nonans.shape

(1000, 1)

In [17]:
log_nonans2 = log.dropna(axis = 0)
log_nonans2.shape

(133, 4)

In [22]:
if log.user_id.isna().sum() != 0:
    log_nonans3 = log.drop('user_id', axis = 1)
else:
    log_nonans3 = log
    
if log_nonans3.time.isna().sum() != 0:
    log_nonans3 = log.drop('time', axis = 1)
else:
    pass
log_nonans3.shape

(1000, 3)

#### Удаление дубликатов

In [23]:
log_nodups = log.drop_duplicates(subset = ['user_id', 'time'])
log_nodups.shape

(986, 4)

In [29]:
log1 = log.dropna(subset=['time'], axis = 0)
log1['time'] = log1['time'].apply(lambda x: reg_time(x))
log1.time = pd.to_datetime(log1.time)
print(log1.time.max().date())
log1['minute'] =  log1['time'].apply(lambda x: x.minute)
print(log1.minute.value_counts().head(1))
log1['month'] =  log1['time'].apply(lambda x: x.month)
print(log1.month.value_counts(ascending = True).head(1))

2019-04-20
36    25
Name: minute, dtype: int64
4    170
Name: month, dtype: int64


/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [34]:
log1['dayofweek'] = log1['time'].apply(lambda x: x.dayofweek)
print(log1[log1['dayofweek'].isin([5,6])].dayofweek.value_counts().sum())

283


/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
# с 0 до 5 часов - ночь, с 6 до 11 - утро, с 12 до 17 - день, с 18 до 23 - вечер.

In [38]:
def timeofday(h):
    if 0<=h<=5:
        return 'ночь'
    elif 6<=h<=11:
        return 'утро'
    elif 12<=h<=17:
        return 'день'
    elif 18<=h<=23:
        return 'вечер'

In [39]:
log1['timeofday'] =  log1['time'].apply(lambda x: timeofday(x.hour))
print(log1.timeofday.value_counts())

ночь     265
утро     253
день     240
вечер    227
Name: timeofday, dtype: int64


/Users/natalia/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
